<a href="https://colab.research.google.com/github/Kinds-of-Intelligence-CFI/comparative-object-permanence/blob/measurement-layout/analysis/measurement-layouts/object_permanence_measurement_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparative Object Permanence Measurement Layouts

Authors: K. Voudouris, J. Burden, J. Hernández-Orallo

## INIT

In [ ]:
!pip install pymc --quiet
!pip install numpy --quiet
!pip install arviz --quiet
!pip install erroranalysis --quiet

In [ ]:
import arviz as az
import erroranalysis as ea
import gc
import graphviz
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pymc as pm
import random as rm
import seaborn as sns

from IPython.display import Image
from scipy import stats
from sklearn.model_selection import train_test_split
from google.colab import files
from pymc import model

print(f"Running on PyMC v{pm.__version__}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load Data

While the repo is private, the links break easily due to hashing of the csv location. You need to go to the csv in the repo, click raw, and copy the new link.

In [ ]:
agents_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/measurement-layout/analysis/measurement-layouts/results_final_clean_agents_wide.csv?token=GHSAT0AAAAAACEGARGR6K4B7PVECGJPPUKYZHM4CXQ'
agent_data = pd.read_csv(agents_url)

children_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/measurement-layout/analysis/measurement-layouts/results_final_clean_children_wide.csv?token=GHSAT0AAAAAACEGARGRH5VCIQ5C4CF45O3GZHM4DBQ'
children_data = pd.read_csv(children_url)

synthetic_agents_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/measurement-layout/analysis/measurement-layouts/results_synthetic_agents_wide.csv?token=GHSAT0AAAAAACEGARGQAF33L3OBPP454MKSZHM4DJQ'
synthetic_agents_data = pd.read_csv(synthetic_agents_url)


## Define the Measurement Layout

In [ ]:
## Helper functions

def logistic(x, L=1, k=1, x_0=0):
  return (L / (1 + np.exp(-k * (x-x_0))))

def logistic999(x, min, max):    # This logistic function ensures that if x is at -(max-min), we get prob 0.001, and if x is at (max-min), we get prob 0.999
  x = x - min
  max = max - min
  x = 6.90675478 * x / max
  return 1 / (1 + np.exp(-x))

def visualAcuityLOMargin(ability, goalSmallness): # must return a value between -inf and inf  (more precisely between -maxVisualAcuityAbility and maxVisualAcuityAbility)
  return ability - goalSmallness   # Goes between -inf to inf, with logodds=0 meaning this would lead to 0.5 chance of success

def SimplePrMargin(ability, binaryFeature): # must return a value between 0 and 1
  return 1-((1-ability)*binaryFeature)  # If binaryFeature is 0 then the margin represents p(success)=1. If binaryFeature = 1 then p(success)=ability

#def flatNavMargin(ability, distanceToGoal, numTurns, allocentricOcclusion): # must return a value between -inf and inf  (more precisely between -maxSpatialAbility and maxSpatialAbility)
def flatNavMargin(ability, distanceToGoal, numTurns):
  #return ability - ((distanceToGoal * numTurns)*allocentricOcclusion)   # Goes between -inf to inf, with logodds=0 meaning this would lead to 0.5 chance of success
  return ability - ((distanceToGoal * numTurns))

def objPermanenceLOMargin(ability, allocentricOcclusion, cvChickP, pctbP, lightsOut, distanceToGoal, numPositions, maxPermAbility, lightsOutPenalisation, uniformAbilitySlack): #,  visualAcuityP): # must return a value between -inf and inf (more precisely between -maxPermAbility and maxPermAbility)
#def objPermanenceLOMargin(ability, allocentricOcclusion, lightsOut, distanceToGoal, numPositions, maxPermAbility, lightsOutPenalisation, uniformAbilitySlack):
  assert lightsOutPenalisation < uniformAbilitySlack and lightsOutPenalisation > 1, "Penalisation for lights out must be lower than the slack on abilities."
  OPPerformance = ability  - ((distanceToGoal * numPositions)*(lightsOut*lightsOutPenalisation)) #this returns a value that is lower when (a) ability is lower, (b) the goal is occluded for longer or there are more positions where it could be occluded, or (c) when the lights go out (by a small penalisation)
  ability = maxPermAbility - ((maxPermAbility-OPPerformance)*allocentricOcclusion) * cvChickP * pctbP # this returns an ability value that is modulated by performance on the different paradigms. The agent needs to be good at both to have a high OP ability
  #return maxPermAbility-((maxPermAbility-OPPerformance)*((allocentricOcclusion*cvChick) + (allocentricOcclusion*pctb))) #multiply by a factor that introduces whether the instance is a an OP CV task or an OP PCTB task (independent, so ((allocentricOcclusion*cvChick) + (allocentricOcclusion*pctb)) should be 0 or 1)
  #return maxPermAbility - ((maxPermAbility-OPPerformance)*allocentricOcclusion)
  return ability

def scaledBeta(name, a, b, min, max, simpleGraph = True):
  if (simpleGraph):
    beta = pm.Beta(name, a, b)
    return beta * (max - min) + min
  else:
    beta = pm.Beta(f"{name}_raw", a, b)
    return pm.Deterministic(name, beta * (max - min) + min)


In [ ]:
def SetupModel(taskResultsAll, uniformAbilitySlack, agent_type, agent_name = None, sample = 500):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests.
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."

  if agent_type == "agent":
    taskResults = taskResultsAll.dropna(subset = [agent_name])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults[agent_name]
  elif agent_type == "child":
    taskResults = taskResultsAll.dropna(subset = ['success'])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults['success']
  else:
    print("Agent not recognised. Quitting.")
    return



  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistance = taskResults["cityBlockDistanceToGoal"].max()
  maxTurns = taskResults["minNumTurnsRequired"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max() * uniformAbilitySlack
  maxPermAbility = ((taskResults["cityBlockDistanceToGoal"] * taskResults["numChoices"]).max()) * uniformAbilitySlack
  maxFlatNav = ((taskResults["cityBlockDistanceToGoal"]*taskResults["minNumTurnsRequired"])).max() * uniformAbilitySlack
  #maxPermAbility = maxChoices * uniformAbilitySlack
  #maxMemoryAbility = maxDistance * uniformAbilitySlack
  #maxPermAbility = 1
  #maxMemoryAbility = 1

  # Decide 'minimum capabilities' based on easiest values in the dataset
  minDistance = taskResults["cityBlockDistanceToGoal"].min()
  minTurns = taskResults["minNumTurnsRequired"].min()
  minChoices = taskResults["numChoices"].min()
  minGoalSize = taskResults["mainGoalSize"].min() * (1-(uniformAbilitySlack-1))
  minPermAbility = ((taskResults["cityBlockDistanceToGoal"] * taskResults["numChoices"]).min()) * (1-(uniformAbilitySlack-1))
  minFlatNav = ((taskResults["cityBlockDistanceToGoal"]*taskResults["minNumTurnsRequired"])).min() * (1-(uniformAbilitySlack-1))
  #minPermAbility = 0

  m = pm.Model()
  with m:

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude
    abilityMin["objPermAbility"] = minPermAbility
    abilityMax["objPermAbility"] = maxPermAbility

    #objPermAbility = scaledBeta("objPermAbility", 1,1, minPermAbility, maxPermAbility, simpleGraph=False)
    abilityMin["objPermAbility"] = minPermAbility
    abilityMax["objPermAbility"] = maxPermAbility

    #memoryAbility = pm.Uniform("memoryAbility", 0, maxMemoryAbility)  # [0,maxMemoryAbility] This is the same as above, but we have the ability in the right magnitude
    #abilityMin["memoryAbility"] = 0
    #abilityMax["memoryAbility"] = maxMemoryAbility

    #Visual acuity
    visualAcuityAbility = pm.Uniform("visualAcuityAbility", minGoalSize, maxGoalSize)
    abilityMin["visualAcuityAbility"] = minGoalSize
    abilityMax["visualAcuityAbility"] = maxGoalSize

    # Flat Navigation Ability
    flatNavAbility = pm.Uniform("flatNavAbility", minFlatNav, maxFlatNav)      # how much navigation is involved, i.e, how far away and how circuitous is the path to the goal?
    #flatNavAbility = pm.Uniform("flatNavAbility", 0, maxTurns*maxDistance)
    abilityMin["flatNavAbility"] = minFlatNav
    #abilityMax["flatNavAbility"] = maxDistance*maxTurns
    abilityMax["flatNavAbility"] = maxFlatNav

    # Lava Ability
    lavaAbility = scaledBeta("lavaAbility", 1,1, 0, 2, simpleGraph=False)
    #lavaAbility = pm.Beta("lavaAbility", 1,1)                        # [0,1] Specific ability with lava
    abilityMin["lavaAbility"] = 0
    abilityMax["lavaAbility"] = 2

    # Ramp Ability
    #rampAbility = pm.Beta("rampAbility",1,1)                          # [0,1] Specific ability with ramps
    #abilityMin["rampAbility"] = 0
    #abilityMax["rampAbility"] = 1

    # Goal Right Ability
    #rightAbility = pm.Beta("rightAbility", 1, 1)
    rightAbility = scaledBeta("rightAbility", 1,1, 0, 2, simpleGraph=False)
    abilityMin["rightAbility"] = 0
    abilityMax["rightAbility"] = 2

    # Goal Left Ability
    #leftAbility = pm.Beta("leftAbility", 1, 1)
    leftAbility = scaledBeta("leftAbility", 1,1, 0, 2, simpleGraph=False)
    abilityMin["leftAbility"] = 0
    abilityMax["leftAbility"] = 2

    # Goal Ahead Ability
    #aheadAbility = pm.Beta("aheadAbility", 1, 1)
    aheadAbility = scaledBeta("aheadAbility", 1,1, 0, 2, simpleGraph=False)
    abilityMin["aheadAbility"] = 0
    abilityMax["aheadAbility"] = 2

    # CV Chick Ability
    #CVChickAbility = pm.Beta("CVChickAbility", 1, 1)
    CVChickAbility = scaledBeta("CVChickAbility", 1,1, 0, 2, simpleGraph=False)
    abilityMin["CVChickAbility"] = 0
    abilityMax["CVChickAbility"] = 2

    # PCTB Ability
    #PCTBAbility = pm.Beta("PCTBAbility", 1, 1)
    PCTBAbility = scaledBeta("PCTBAbility", 1,1, 0, 2, simpleGraph=False)
    abilityMin["PCTBAbility"] = 0
    abilityMax["PCTBAbility"] = 2

    # Lights Out Ability
    #LOAbility = pm.Beta("lightsOutAbility", 1, 1)
    LOAbility = scaledBeta("lightsOutAbility", 1,1, 0, 2, simpleGraph=False)
    abilityMin["lightsOutAbility"] = 0
    abilityMax["lightsOutAbility"] = 2



    ## Environment variables as Deterministic (about the instance)

    lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values)
    #rampPresence = pm.MutableData("rampPresence", taskResults["taskCriticalRampPresence"].values)
    lightsOutPresence = pm.MutableData("lightsOutPresence", taskResults["lightsOutPresence"].values)
    #numGoals = pm.MutableData("numberOfGoals", taskResults["numGoalsAll"].values)
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values)
    goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values)
    goalDist = pm.MutableData("goalDistance", taskResults["cityBlockDistanceToGoal"].values)
    numTurns = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsRequired"])
    goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"])
    goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"])
    goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"])
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"].values)
    CVTest = pm.MutableData("CVChickTest", taskResults["cvchickTask"])
    PCTBTest = pm.MutableData("PCTBTest", taskResults["pctbTask"])

    ## Margins

    goalSmallness = maxGoalSize - goalSize
    visualAcuityP = pm.Deterministic("visualAcuityP", logistic999(visualAcuityLOMargin(visualAcuityAbility, goalSmallness), min = minGoalSize, max = maxGoalSize))

    rightP = pm.Deterministic("rightPerformance", logistic999(SimplePrMargin(rightAbility, goalRight), min = 0, max = 2))
    aheadP = pm.Deterministic("aheadPerformance", logistic999(SimplePrMargin(aheadAbility, goalAhead), min = 0, max = 2))
    leftP = pm.Deterministic("leftPerformance", logistic999(SimplePrMargin(leftAbility, goalLeft), min = 0, max = 2))

    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavMargin(flatNavAbility, goalDist, numTurns, opTest)))
    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavMargin(flatNavAbility, goalDist, numTurns)))
    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavAbility - (numTurns * goalDist)))
    flatNavP = pm.Deterministic("flatNavP", logistic999(flatNavAbility - (numTurns * goalDist), min = minFlatNav, max = maxFlatNav))

    lavaP = pm.Deterministic("lavaP", logistic999(SimplePrMargin(lavaAbility, lavaPresence), min = 0, max = 2))

    lightsOutP = pm.Deterministic("lightsOutP", logistic999(SimplePrMargin(LOAbility, lightsOutPresence), min = 0, max = 2))

    #rampP = pm.Deterministic("rampP", SimplePrMargin(rampAbility, rampPresence))

    cvchickP = pm.Deterministic("cvChickP", logistic999(SimplePrMargin(CVChickAbility, CVTest), min = 0, max = 2))
    pctbP = pm.Deterministic("pctbP", logistic999(SimplePrMargin(PCTBAbility, PCTBTest), min = 0, max = 2))

    #navP = pm.Deterministic("navP", (flatNavP * lavaP * rampP * rightP * aheadP * leftP * cvchickP * pctbP)) #, visualAcuityP))
    navP = pm.Deterministic("navP", (flatNavP * lavaP * rightP * aheadP * leftP))

    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, CVTest, PCTBTest, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)
    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)
    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, cvchickP, pctbP, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)

    #OPPerformance = (objPermAbility  - ((goalDist * numChoices)*opTest)) #this returns a value that is lower when (a) ability is lower, (b) the goal is occluded for longer or there are more positions where it could be occluded, or (c) when the lights go out (by a small penalisation)
    #OPPerformance = (objPermAbility  - ((goalDist * numChoices)))
    #OPPerformance = (objPermAbility  - numChoices)
    #OPPerformance = (objPermAbility  - ((goalDist*opTest) * (numChoices*opTest)))
    #OPPerformance = objPermAbility  - ((goalDist * numChoices)*opTest)
    #OPability = maxPermAbility - ((maxPermAbility-OPPerformance)*opTest)
    #objPermP = pm.Deterministic("objPermP", logistic(OPLOM))
    #objPermP = pm.Deterministic("objPermP", logistic(OPPerformance, x_0 = ((maxPermAbility - minPermAbility)/2)+minPermAbility))
    #objPermP = pm.Deterministic("objPermP", logistic(SimplePrMargin(objPermAbility, opTest), x_0 = ((maxPermAbility - minPermAbility)/2)+minPermAbility))
    OPPerformance = (objPermAbility  - (((goalDist*opTest) * numChoices) + (numChoices * (1-opTest))))
    #objPermP = pm.Deterministic("objPermP", logistic(OPPerformance, x_0 = ((maxPermAbility - minPermAbility)/2)+minPermAbility))
    #objPermP = pm.Deterministic("objPermP", logistic(OPPerformance))
    objPermP = pm.Deterministic("objPermP", (logistic999(OPPerformance, min = minPermAbility, max = maxPermAbility) * cvchickP * pctbP * lightsOutP))

    #flatNavP = pm.Deterministic("flatNavP", logistic((flatNavAbility - (numTurns * goalDist)), x_0 = ((maxFlatNav - minFlatNav)/2)+minFlatNav))
    noise = 1 - np.mean(results)  # With this noise is complementary to result prior.
    noisePar = pm.Uniform("noisePar", 0, 1)
    finalP = pm.Deterministic("finalP", (1-noisePar)*(objPermP * navP * visualAcuityP)+(noisePar*noise))

    #finalP = pm.Deterministic("finalP", (objPermP * flatNavP))
    #finalP = pm.Deterministic("finalP", (1-((1-objPermP)*(1-flatNavP)))) #compensatory finalPerformance
    #finalP = pm.Deterministic("finalP", (objPermP * flatNavP * visualAcuityP))
    #finalP = pm.Deterministic("finalP", (objPermP * navP * visualAcuityP))

    #taskPerformance = pm.Bernoulli("taskPerformance", flatNavP, observed=results)
    taskPerformance = pm.Bernoulli("taskPerformance", finalP, observed=results)
    #taskPerformance = pm.Bernoulli("taskPerformance", objPermP, observed=results)
  #pm.model_graph.model_to_graphviz(m)

  return m, abilityMin, abilityMax

In [ ]:
m, min, max = SetupModel(synthetic_agents_data, uniformAbilitySlack=1, agent_type='agent', agent_name="perfectAgent", sample = None)
gv = pm.model_graph.model_to_graphviz(m)
gv.format = "png"
gv.render(filename='viz/FullMeasurementLayout.gv', directory='/content/drive/Shareddrives/comparative-object-permanence')
Image("/content/drive/Shareddrives/comparative-object-permanence/FullMeasurementLayout.gv.png")
gv

In [ ]:
# A function for pulling out the means and standard deviations for abilities of interest
def analyzeAgentResults(agentName, agentData, abilitiesToShow):

  #abilityMeans = [] # empty list to add in ability means to
  #abilitySDs = []

  abilityMeans = {}
  abilitySDs = {}

  for a in abilitiesToShow: #iterate through each ability, add posterior mean to dataframe, and plot posterior

    posteriorMean = float(np.mean(agentData['posterior'][a])) # calculate posterior mean
    posteriorSD = float(np.std(agentData['posterior'][a])) #calculate posterior sd
    abilityMeans[a] = posteriorMean
    abilitySDs[a] = posteriorSD
    #abilityMeans.append(posteriorMean)
    #abilitySDs.append(posteriorSD)

  return abilityMeans, abilitySDs

# A function for padding the testing data with 0s for prediction
def pad(testingData, trainingDataSize):
    return testingData.append(pd.Series(np.zeros(trainingDataSize-len(testingData), dtype=int)))

# A function for making predictions based on a fitted measurement layout
def predict(m, agentData, dfTest, agent, len_training):
  with m:
    # set the data for prediction
    pm.set_data({"lavaPresence": pad(dfTest["lavaPresence"], len_training)})
    pm.set_data({"lightsOutPresence": pad(dfTest["lightsOutPresence"], len_training)})
    #pm.set_data({"rampPresence": pad(dfTest["taskCriticalRampPresence"], len_training)})
    pm.set_data({"numChoices": pad(dfTest["numChoices"], len_training)})
    pm.set_data({"minTurnsToGoal": pad(dfTest["minNumTurnsRequired"], len_training)})
    pm.set_data({"goalDistance": pad(dfTest["cityBlockDistanceToGoal"], len_training)})
    pm.set_data({"goalSize": pad(dfTest["mainGoalSize"], len_training)})
    pm.set_data({"goalRight": pad(dfTest["goalRightRelToStart"], len_training)})
    pm.set_data({"goalAhead": pad(dfTest["goalCentreRelToStart"], len_training)})
    pm.set_data({"goalLeft": pad(dfTest["goalLeftRelToStart"], len_training)})
    pm.set_data({"CVChickTest": pad(dfTest["cvchickTask"], len_training)})
    pm.set_data({"PCTBTest": pad(dfTest["pctbTask"], len_training)})
    pm.set_data({"allocentricOPTest": pad(dfTest["goalBecomesAllocentricallyOccluded"], len_training)})

    predictions=pm.sample_posterior_predictive(agentData, var_names=["finalP"], return_inferencedata=False,predictions=True,extend_inferencedata=False)
    predictionChainRuns =predictions["finalP"][:,:,0:len(dfTest)]
    predictionsInstance = np.mean(predictionChainRuns, (0,1))

    return predictionsInstance,  dfTest[agent].to_numpy()


def brierScore(preds, outs):
    return 1/len(preds) * sum( (preds-outs)**2 )

def brierDecomp(preds, outs):

  brier= 1/len(preds) * sum( (preds-outs)**2 )
  ## bin predictions
  bins = np.linspace(0,1,11)
  binCenters = (bins[:-1] +bins[1:]) /2
  binPredInds = np.digitize(preds,binCenters)
  binnedPreds = bins[binPredInds]

  binTrueFreqs = np.zeros(10)
  binPredFreqs = np.zeros(10)
  binCounts = np.zeros(10)

  for i in range(10):
      idx = (preds >= bins[i]) & (preds < bins[i+1])

      binTrueFreqs[i] = np.sum(outs[idx])/np.sum(idx) if np.sum(idx) > 0 else 0
     # print(np.sum(outs[idx]), np.sum(idx), binTrueFreqs[i])
      binPredFreqs[i] = np.mean(preds[idx]) if np.sum(idx) > 0 else 0
      binCounts[i] = np.sum(idx)

  calibration = np.sum(binCounts * (binTrueFreqs - binPredFreqs) ** 2) / np.sum(binCounts) if np.sum(binCounts)>0 else 0
  refinement = np.sum(binCounts * (binTrueFreqs *(1 - binTrueFreqs))) / np.sum(binCounts) if np.sum(binCounts)> 0 else 0
  # Compute refinement component
  #refinement = brier - calibration
  return brier, calibration, refinement



## Run Complete Measurement Layout

In [ ]:
real_agent_names = ["Random_Walker_Fixed_Forwards_Saccade_15_Angle_10_356",
                    "Random_Walker_Fixed_Forwards_Saccade_15_Angle_10_1997",
                    "Random_Walker_Fixed_Forwards_Saccade_15_Angle_10_2023",
                    "Random_Walker_Fixed_Forwards_Saccade_15_Angle_10_1815",
                    "Random_Walker_Fixed_Forwards_Saccade_15_Angle_10_3761",
                    "Random_Walker_Rayleigh_Flight_Norm_Saccade_mu_15_sig_5_Uniform_Angles_1_to_60_356",
                    "Random_Walker_Rayleigh_Flight_Norm_Saccade_mu_15_sig_5_Uniform_Angles_1_to_60_1997",
                    "Random_Walker_Rayleigh_Flight_Norm_Saccade_mu_15_sig_5_Uniform_Angles_1_to_60_2023",
                    "Random_Walker_Rayleigh_Flight_Norm_Saccade_mu_15_sig_5_Uniform_Angles_1_to_60_1815",
                    "Random_Walker_Rayleigh_Flight_Norm_Saccade_mu_15_sig_5_Uniform_Angles_1_to_60_3761",
                    "Random_Walker_Levy_Flight_Cauchy_Saccade_mode_15_Uniform_Angles_1_to_60_356",
                    "Random_Walker_Levy_Flight_Cauchy_Saccade_mode_15_Uniform_Angles_1_to_60_1997",
                    "Random_Walker_Levy_Flight_Cauchy_Saccade_mode_15_Uniform_Angles_1_to_60_2023",
                    "Random_Walker_Levy_Flight_Cauchy_Saccade_mode_15_Uniform_Angles_1_to_60_1815",
                    "Random_Walker_Levy_Flight_Cauchy_Saccade_mode_15_Uniform_Angles_1_to_60_3761",
                    "Random_Action_Agent_no_bias_no_correlation_uniform_step_length_max_20_356",
                    "Random_Action_Agent_no_bias_no_correlation_uniform_step_length_max_20_1997",
                    "Random_Action_Agent_no_bias_no_correlation_uniform_step_length_max_20_2023",
                    "Random_Action_Agent_no_bias_no_correlation_uniform_step_length_max_20_1815",
                    "Random_Action_Agent_no_bias_no_correlation_uniform_step_length_max_20_3761",
                    "Random_Action_Agent_cephalocaudal_bias_cauchy_step_length_mode_15_356",
                    "Random_Action_Agent_cephalocaudal_bias_cauchy_step_length_mode_15_1997",
                    "Random_Action_Agent_cephalocaudal_bias_cauchy_step_length_mode_15_2023",
                    "Random_Action_Agent_cephalocaudal_bias_cauchy_step_length_mode_15_1815",
                    "Random_Action_Agent_cephalocaudal_bias_cauchy_step_length_mode_15_3761",
                    "Vanilla_Braitenberg_15_rays_over_60_degs_356",
                    "Vanilla_Braitenberg_15_rays_over_60_degs_1997",
                    "Vanilla_Braitenberg_15_rays_over_60_degs_2023",
                    "Vanilla_Braitenberg_15_rays_over_60_degs_1815",
                    "Vanilla_Braitenberg_15_rays_over_60_degs_3761"]

synthetic_agent_names = ["perfectAgent",
                         "failedAgent",
                         "noOPAgent",
                         "lowVisualAcuityAgent",
                         "poorNavigationOPAgent",
                         "CVChickBasicSpecificAgent",
                         "PCTBBasicSpecificAgent"]

In [ ]:
synthetic_agents_training, synthetic_agents_test = train_test_split(synthetic_agents_data, test_size = 0.2)
real_agents_training, real_agents_test = train_test_split(agent_data, test_size=0.2)
children_training, children_test = train_test_split(children_data, test_size=0.2)


In [ ]:
abilitiesToShow = ["objPermAbility", "visualAcuityAbility", "flatNavAbility", "lavaAbility", "rightAbility", "leftAbility", "aheadAbility", "CVChickAbility", "PCTBAbility", "lightsOutAbility", "noisePar"]

pymc_sample_num = 2000

slack = 1

In [ ]:
agentName = []
modelBrierScore = []
aggregateBrierScore = []
modelCalibration = []
aggregateCalibration = []
modelRefinement = []
aggregateRefinement = []
modelBetterThanAgg = []
meanSuccess = []
OPAbilityAllMean = []
navAbilityAllMean = []
visualAcuityAbilityAllMean = []
flatNavAbilityAllMean = []
lavaAbilityAllMean = []
rightAbilityAllMean = []
leftAbilityAllMean = []
aheadAbilityAllMean = []
CVChickAbilityAllMean = []
PCTBAbilityAllMean = []
lightsOutAbilityAllMean = []
noiseParAllMean = []
OPAbilityAllSD = []
navAbilityAllSD = []
visualAcuityAbilityAllSD = []
flatNavAbilityAllSD = []
lavaAbilityAllSD = []
rightAbilityAllSD = []
leftAbilityAllSD = []
aheadAbilityAllSD = []
CVChickAbilityAllSD = []
PCTBAbilityAllSD = []
lightsOutAbilityAllSD = []
noiseParAllSD = []
OPAbilityAllMin = []
navAbilityAllMin = []
visualAcuityAbilityAllMin = []
flatNavAbilityAllMin = []
lavaAbilityAllMin = []
rightAbilityAllMin = []
leftAbilityAllMin = []
aheadAbilityAllMin = []
CVChickAbilityAllMin = []
PCTBAbilityAllMin = []
lightsOutAbilityAllMin = []
noiseParAllMin = []
OPAbilityAllMax = []
navAbilityAllMax = []
visualAcuityAbilityAllMax = []
flatNavAbilityAllMax = []
lavaAbilityAllMax = []
rightAbilityAllMax = []
leftAbilityAllMax = []
aheadAbilityAllMax = []
CVChickAbilityAllMax = []
PCTBAbilityAllMax = []
lightsOutAbilityAllMax = []
noiseParAllMax = []

In [ ]:
for agent in synthetic_agent_names:
  model_train, min, max = SetupModel(synthetic_agents_training, uniformAbilitySlack=slack, agent_type='agent', agent_name=agent, sample = None)
  with model_train:
    data_training = pm.sample(pymc_sample_num, target_accept=0.95)

  predictions, testOutputs = predict(model_train, data_training, synthetic_agents_test, agent, len(synthetic_agents_training))
  agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
  agentAggBrierScore, agentAggCalibration, agentAggRefinement = brierDecomp(np.repeat(np.mean(synthetic_agents_training[agent]), len(testOutputs)), testOutputs)
  aggregateBrierScore.append(agentAggBrierScore)
  aggregateCalibration.append(agentAggCalibration)
  aggregateRefinement.append(agentAggRefinement)
  modelBrierScore.append(agentBrierScore)
  modelCalibration.append(agentCalibration)
  modelRefinement.append(agentRefinement)
  if agentBrierScore < agentAggBrierScore:
    modelBetterThanAgg.append(True)
  else:
    modelBetterThanAgg.append(False)

  model_all, min, max = SetupModel(synthetic_agents_data, uniformAbilitySlack=slack, agent_type='agent', agent_name=agent, sample = None)
  with model_all:
    data_all = pm.sample(pymc_sample_num, target_accept=0.95)
  mu, sd  = analyzeAgentResults(agent, data_all, abilitiesToShow)

  OPAbilityAllMean.append(mu["objPermAbility"])
  visualAcuityAbilityAllMean.append(mu["visualAcuityAbility"])
  flatNavAbilityAllMean.append(mu["flatNavAbility"])
  lavaAbilityAllMean.append(mu["lavaAbility"])
  rightAbilityAllMean.append(mu["rightAbility"])
  leftAbilityAllMean.append(mu["leftAbility"])
  aheadAbilityAllMean.append(mu["aheadAbility"])
  CVChickAbilityAllMean.append(mu["CVChickAbility"])
  PCTBAbilityAllMean.append(mu["PCTBAbility"])
  lightsOutAbilityAllMean.append(mu["lightsOutAbility"])
  noiseParAllMean.append(mu["noisePar"])
  OPAbilityAllSD.append(sd["objPermAbility"])
  visualAcuityAbilityAllSD.append(sd["visualAcuityAbility"])
  flatNavAbilityAllSD.append(sd["flatNavAbility"])
  lavaAbilityAllSD.append(sd["lavaAbility"])
  rightAbilityAllSD.append(sd["rightAbility"])
  leftAbilityAllSD.append(sd["leftAbility"])
  aheadAbilityAllSD.append(sd["aheadAbility"])
  CVChickAbilityAllSD.append(sd["CVChickAbility"])
  PCTBAbilityAllSD.append(sd["PCTBAbility"])
  lightsOutAbilityAllSD.append(sd["lightsOutAbility"])
  noiseParAllSD.append(sd["noisePar"])
  OPAbilityAllMin.append(min["objPermAbility"])
  visualAcuityAbilityAllMin.append(min["visualAcuityAbility"])
  flatNavAbilityAllMin.append(min["flatNavAbility"])
  lavaAbilityAllMin.append(min["lavaAbility"])
  rightAbilityAllMin.append(min["rightAbility"])
  leftAbilityAllMin.append(min["leftAbility"])
  aheadAbilityAllMin.append(min["aheadAbility"])
  CVChickAbilityAllMin.append(min["CVChickAbility"])
  PCTBAbilityAllMin.append(min["PCTBAbility"])
  lightsOutAbilityAllMin.append(min["lightsOutAbility"])
  noiseParAllMin.append(0)
  OPAbilityAllMax.append(max["objPermAbility"])
  visualAcuityAbilityAllMax.append(max["visualAcuityAbility"])
  flatNavAbilityAllMax.append(max["flatNavAbility"])
  lavaAbilityAllMax.append(max["lavaAbility"])
  rightAbilityAllMax.append(max["rightAbility"])
  leftAbilityAllMax.append(max["leftAbility"])
  aheadAbilityAllMax.append(max["aheadAbility"])
  CVChickAbilityAllMax.append(max["CVChickAbility"])
  PCTBAbilityAllMax.append(max["PCTBAbility"])
  lightsOutAbilityAllMax.append(max["lightsOutAbility"])
  noiseParAllMax.append(1)

  meanSuccess.append(np.mean(synthetic_agents_data[agent]))
  agentName.append(agent)

  finalDF = pd.DataFrame({"Agent Name": agentName,
                          "Model Brier Score":modelBrierScore,
                          "Aggregate Brier Scores": aggregateBrierScore,
                          "Model Calibration": modelCalibration,
                          "Aggregate Calibration":aggregateCalibration,
                          "Model Refinement":modelRefinement,
                          "Aggregate Refinement":aggregateRefinement,
                          "Model Better? (Based on Brier Score)":modelBetterThanAgg,
                          "Success Score":meanSuccess,
                          "Object Permanence Ability Mean (All)": OPAbilityAllMean,
                          "Visual Acuity Ability Mean (All)" : visualAcuityAbilityAllMean,
                          "Flat Navigation Ability Mean (All)" : flatNavAbilityAllMean,
                          "Lava Ability Mean (All)" : lavaAbilityAllMean,
                          "Right Ability Mean (All)" : rightAbilityAllMean,
                          "Left Ability Mean (All)" : leftAbilityAllMean,
                          "Ahead Ability Mean (All)" : aheadAbilityAllMean,
                          "CVChick Ability Mean (All)" : CVChickAbilityAllMean,
                          "PCTB Ability Mean (All)" :  PCTBAbilityAllMean,
                          "Lights Out Ability Mean (All)" : lightsOutAbilityAllMean,
                          "Noise Parameter Mean (All)" : noiseParAllMean,
                          "Object Permanence Ability SD (All)": OPAbilityAllSD,
                          "Visual Acuity Ability SD (All)" : visualAcuityAbilityAllSD,
                          "Flat Navigation Ability SD (All)" : flatNavAbilityAllSD,
                          "Lava Ability SD (All)" : lavaAbilityAllSD,
                          "Right Ability SD (All)" : rightAbilityAllSD,
                          "Left Ability SD (All)" : leftAbilityAllSD,
                          "Ahead Ability SD (All)" : aheadAbilityAllSD,
                          "CVChick Ability SD (All)" : CVChickAbilityAllSD,
                          "PCTB Ability SD (All)" :  PCTBAbilityAllSD,
                          "Lights Out Ability SD (All)" : lightsOutAbilityAllSD,
                          "Noise Parameter SD (All)" : noiseParAllSD,
                          "Object Permanence Ability Min (All)": OPAbilityAllMin,
                          "Visual Acuity Ability Min (All)" : visualAcuityAbilityAllMin,
                          "Flat Navigation Ability Min (All)" : flatNavAbilityAllMin,
                          "Lava Ability Min (All)" : lavaAbilityAllMin,
                          "Right Ability Min (All)" : rightAbilityAllMin,
                          "Left Ability Min (All)" : leftAbilityAllMin,
                          "Ahead Ability Min (All)" : aheadAbilityAllMin,
                          "CVChick Ability Min (All)" : CVChickAbilityAllMin,
                          "PCTB Ability Min (All)" :  PCTBAbilityAllMin,
                          "Lights Out Ability Min (All)" : lightsOutAbilityAllMin,
                          "Noise Parameter Min (All)" : noiseParAllMin,
                          "Object Permanence Ability Max (All)": OPAbilityAllMax,
                          "Visual Acuity Ability Max (All)" : visualAcuityAbilityAllMax,
                          "Flat Navigation Ability Max (All)" : flatNavAbilityAllMax,
                          "Lava Ability Max (All)" : lavaAbilityAllMax,
                          "Right Ability Max (All)" : rightAbilityAllMax,
                          "Left Ability Max (All)" : leftAbilityAllMax,
                          "Ahead Ability Max (All)" : aheadAbilityAllMax,
                          "CVChick Ability Max (All)" : CVChickAbilityAllMax,
                          "PCTB Ability Max (All)" :  PCTBAbilityAllMax,
                          "Lights Out Ability Max (All)" : lightsOutAbilityAllMax,
                          "Noise Parameter Max (All)" : noiseParAllMax})
  finalDF.to_csv("/content/drive/Shareddrives/comparative-object-permanence/finalResults.csv")


for agent in real_agent_names:
  model_train, min, max = SetupModel(real_agents_training, uniformAbilitySlack=slack, agent_type='agent', agent_name=agent, sample = None)
  with model_train:
    data_training = pm.sample(pymc_sample_num, target_accept=0.95)

  predictions, testOutputs = predict(model_train, data_training, real_agents_test, agent, len(real_agents_training))
  agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
  agentAggBrierScore, agentAggCalibration, agentAggRefinement = brierDecomp(np.repeat(np.mean(real_agents_training[agent]), len(testOutputs)), testOutputs)
  aggregateBrierScore.append(agentAggBrierScore)
  aggregateCalibration.append(agentAggCalibration)
  aggregateRefinement.append(agentAggRefinement)
  modelBrierScore.append(agentBrierScore)
  modelCalibration.append(agentCalibration)
  modelRefinement.append(agentRefinement)
  if agentBrierScore < agentAggBrierScore:
    modelBetterThanAgg.append(True)
  else:
    modelBetterThanAgg.append(False)

  model_all, min, max = SetupModel(agent_data, uniformAbilitySlack=slack, agent_type='agent', agent_name=agent, sample = None)
  with model_all:
    data_all = pm.sample(pymc_sample_num, target_accept=0.95)
  mu, sd  = analyzeAgentResults(agent, data_all, abilitiesToShow)

  OPAbilityAllMean.append(mu["objPermAbility"])
  visualAcuityAbilityAllMean.append(mu["visualAcuityAbility"])
  flatNavAbilityAllMean.append(mu["flatNavAbility"])
  lavaAbilityAllMean.append(mu["lavaAbility"])
  rightAbilityAllMean.append(mu["rightAbility"])
  leftAbilityAllMean.append(mu["leftAbility"])
  aheadAbilityAllMean.append(mu["aheadAbility"])
  CVChickAbilityAllMean.append(mu["CVChickAbility"])
  PCTBAbilityAllMean.append(mu["PCTBAbility"])
  lightsOutAbilityAllMean.append(mu["lightsOutAbility"])
  noiseParAllMean.append(mu["noisePar"])
  OPAbilityAllSD.append(sd["objPermAbility"])
  visualAcuityAbilityAllSD.append(sd["visualAcuityAbility"])
  flatNavAbilityAllSD.append(sd["flatNavAbility"])
  lavaAbilityAllSD.append(sd["lavaAbility"])
  rightAbilityAllSD.append(sd["rightAbility"])
  leftAbilityAllSD.append(sd["leftAbility"])
  aheadAbilityAllSD.append(sd["aheadAbility"])
  CVChickAbilityAllSD.append(sd["CVChickAbility"])
  PCTBAbilityAllSD.append(sd["PCTBAbility"])
  lightsOutAbilityAllSD.append(sd["lightsOutAbility"])
  noiseParAllSD.append(sd["noisePar"])
  OPAbilityAllMin.append(min["objPermAbility"])
  visualAcuityAbilityAllMin.append(min["visualAcuityAbility"])
  flatNavAbilityAllMin.append(min["flatNavAbility"])
  lavaAbilityAllMin.append(min["lavaAbility"])
  rightAbilityAllMin.append(min["rightAbility"])
  leftAbilityAllMin.append(min["leftAbility"])
  aheadAbilityAllMin.append(min["aheadAbility"])
  CVChickAbilityAllMin.append(min["CVChickAbility"])
  PCTBAbilityAllMin.append(min["PCTBAbility"])
  lightsOutAbilityAllMin.append(min["lightsOutAbility"])
  noiseParAllMin.append(0)
  OPAbilityAllMax.append(max["objPermAbility"])
  visualAcuityAbilityAllMax.append(max["visualAcuityAbility"])
  flatNavAbilityAllMax.append(max["flatNavAbility"])
  lavaAbilityAllMax.append(max["lavaAbility"])
  rightAbilityAllMax.append(max["rightAbility"])
  leftAbilityAllMax.append(max["leftAbility"])
  aheadAbilityAllMax.append(max["aheadAbility"])
  CVChickAbilityAllMax.append(max["CVChickAbility"])
  PCTBAbilityAllMax.append(max["PCTBAbility"])
  lightsOutAbilityAllMax.append(max["lightsOutAbility"])
  noiseParAllMax.append(1)

  meanSuccess.append(np.mean(agent_data[agent]))
  agentName.append(agent)

  finalDF = pd.DataFrame({"Agent Name": agentName,
                          "Model Brier Score":modelBrierScore,
                          "Aggregate Brier Scores": aggregateBrierScore,
                          "Model Calibration": modelCalibration,
                          "Aggregate Calibration":aggregateCalibration,
                          "Model Refinement":modelRefinement,
                          "Aggregate Refinement":aggregateRefinement,
                          "Model Better? (Based on Brier Score)":modelBetterThanAgg,
                          "Success Score":meanSuccess,
                          "Object Permanence Ability Mean (All)": OPAbilityAllMean,
                          "Visual Acuity Ability Mean (All)" : visualAcuityAbilityAllMean,
                          "Flat Navigation Ability Mean (All)" : flatNavAbilityAllMean,
                          "Lava Ability Mean (All)" : lavaAbilityAllMean,
                          "Right Ability Mean (All)" : rightAbilityAllMean,
                          "Left Ability Mean (All)" : leftAbilityAllMean,
                          "Ahead Ability Mean (All)" : aheadAbilityAllMean,
                          "CVChick Ability Mean (All)" : CVChickAbilityAllMean,
                          "PCTB Ability Mean (All)" :  PCTBAbilityAllMean,
                          "Lights Out Ability Mean (All)" : lightsOutAbilityAllMean,
                          "Noise Parameter Mean (All)" : noiseParAllMean,
                          "Object Permanence Ability SD (All)": OPAbilityAllSD,
                          "Visual Acuity Ability SD (All)" : visualAcuityAbilityAllSD,
                          "Flat Navigation Ability SD (All)" : flatNavAbilityAllSD,
                          "Lava Ability SD (All)" : lavaAbilityAllSD,
                          "Right Ability SD (All)" : rightAbilityAllSD,
                          "Left Ability SD (All)" : leftAbilityAllSD,
                          "Ahead Ability SD (All)" : aheadAbilityAllSD,
                          "CVChick Ability SD (All)" : CVChickAbilityAllSD,
                          "PCTB Ability SD (All)" :  PCTBAbilityAllSD,
                          "Lights Out Ability SD (All)" : lightsOutAbilityAllSD,
                          "Noise Parameter SD (All)" : noiseParAllSD,
                          "Object Permanence Ability Min (All)": OPAbilityAllMin,
                          "Visual Acuity Ability Min (All)" : visualAcuityAbilityAllMin,
                          "Flat Navigation Ability Min (All)" : flatNavAbilityAllMin,
                          "Lava Ability Min (All)" : lavaAbilityAllMin,
                          "Right Ability Min (All)" : rightAbilityAllMin,
                          "Left Ability Min (All)" : leftAbilityAllMin,
                          "Ahead Ability Min (All)" : aheadAbilityAllMin,
                          "CVChick Ability Min (All)" : CVChickAbilityAllMin,
                          "PCTB Ability Min (All)" :  PCTBAbilityAllMin,
                          "Lights Out Ability Min (All)" : lightsOutAbilityAllMin,
                          "Noise Parameter Min (All)" : noiseParAllMin,
                          "Object Permanence Ability Max (All)": OPAbilityAllMax,
                          "Visual Acuity Ability Max (All)" : visualAcuityAbilityAllMax,
                          "Flat Navigation Ability Max (All)" : flatNavAbilityAllMax,
                          "Lava Ability Max (All)" : lavaAbilityAllMax,
                          "Right Ability Max (All)" : rightAbilityAllMax,
                          "Left Ability Max (All)" : leftAbilityAllMax,
                          "Ahead Ability Max (All)" : aheadAbilityAllMax,
                          "CVChick Ability Max (All)" : CVChickAbilityAllMax,
                          "PCTB Ability Max (All)" :  PCTBAbilityAllMax,
                          "Lights Out Ability Max (All)" : lightsOutAbilityAllMax,
                          "Noise Parameter Max (All)" : noiseParAllMax})
  finalDF.to_csv("/content/drive/Shareddrives/comparative-object-permanence/finalResults.csv")

model_train, min, max = SetupModel(children_training, uniformAbilitySlack=slack, agent_type='agent', agent_name=agent, sample = None)
with model_train:
  data_training = pm.sample(pymc_sample_num, target_accept=0.95)

predictions, testOutputs = predict(model_train, data_training, children_test, agent, len(children_training))
agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
agentAggBrierScore, agentAggCalibration, agentAggRefinement = brierDecomp(np.repeat(np.mean(children_training[agent]), len(testOutputs)), testOutputs)
aggregateBrierScore.append(agentAggBrierScore)
aggregateCalibration.append(agentAggCalibration)
aggregateRefinement.append(agentAggRefinement)
modelBrierScore.append(agentBrierScore)
modelCalibration.append(agentCalibration)
modelRefinement.append(agentRefinement)
if agentBrierScore < agentAggBrierScore:
  modelBetterThanAgg.append(True)
else:
  modelBetterThanAgg.append(False)

model_all, min, max = SetupModel(children_data, uniformAbilitySlack=slack, agent_type='child', sample = None)
with model_all:
  data_all = pm.sample(pymc_sample_num, target_accept=0.95)
mu, sd  = analyzeAgentResults("child", data_all, abilitiesToShow)

OPAbilityAllMean.append(mu["objPermAbility"])
visualAcuityAbilityAllMean.append(mu["visualAcuityAbility"])
flatNavAbilityAllMean.append(mu["flatNavAbility"])
lavaAbilityAllMean.append(mu["lavaAbility"])
rightAbilityAllMean.append(mu["rightAbility"])
leftAbilityAllMean.append(mu["leftAbility"])
aheadAbilityAllMean.append(mu["aheadAbility"])
CVChickAbilityAllMean.append(mu["CVChickAbility"])
PCTBAbilityAllMean.append(mu["PCTBAbility"])
lightsOutAbilityAllMean.append(mu["lightsOutAbility"])
noiseParAllMean.append(mu["noisePar"])
OPAbilityAllSD.append(sd["objPermAbility"])
visualAcuityAbilityAllSD.append(sd["visualAcuityAbility"])
flatNavAbilityAllSD.append(sd["flatNavAbility"])
lavaAbilityAllSD.append(sd["lavaAbility"])
rightAbilityAllSD.append(sd["rightAbility"])
leftAbilityAllSD.append(sd["leftAbility"])
aheadAbilityAllSD.append(sd["aheadAbility"])
CVChickAbilityAllSD.append(sd["CVChickAbility"])
PCTBAbilityAllSD.append(sd["PCTBAbility"])
lightsOutAbilityAllSD.append(sd["lightsOutAbility"])
noiseParAllSD.append(sd["noisePar"])
OPAbilityAllMin.append(min["objPermAbility"])
visualAcuityAbilityAllMin.append(min["visualAcuityAbility"])
flatNavAbilityAllMin.append(min["flatNavAbility"])
lavaAbilityAllMin.append(min["lavaAbility"])
rightAbilityAllMin.append(min["rightAbility"])
leftAbilityAllMin.append(min["leftAbility"])
aheadAbilityAllMin.append(min["aheadAbility"])
CVChickAbilityAllMin.append(min["CVChickAbility"])
PCTBAbilityAllMin.append(min["PCTBAbility"])
lightsOutAbilityAllMin.append(min["lightsOutAbility"])
noiseParAllMin.append(0)
OPAbilityAllMax.append(max["objPermAbility"])
visualAcuityAbilityAllMax.append(max["visualAcuityAbility"])
flatNavAbilityAllMax.append(max["flatNavAbility"])
lavaAbilityAllMax.append(max["lavaAbility"])
rightAbilityAllMax.append(max["rightAbility"])
leftAbilityAllMax.append(max["leftAbility"])
aheadAbilityAllMax.append(max["aheadAbility"])
CVChickAbilityAllMax.append(max["CVChickAbility"])
PCTBAbilityAllMax.append(max["PCTBAbility"])
lightsOutAbilityAllMax.append(max["lightsOutAbility"])
noiseParAllMax.append(1)

meanSuccess.append(np.mean(children_data['success']))
agentName.append('children')

finalDF = pd.DataFrame({"Agent Name": agentName,
                        "Model Brier Score":modelBrierScore,
                        "Aggregate Brier Scores": aggregateBrierScore,
                        "Model Calibration": modelCalibration,
                        "Aggregate Calibration":aggregateCalibration,
                        "Model Refinement":modelRefinement,
                        "Aggregate Refinement":aggregateRefinement,
                        "Model Better? (Based on Brier Score)":modelBetterThanAgg,
                        "Success Score":meanSuccess,
                        "Object Permanence Ability Mean (All)": OPAbilityAllMean,
                        "Visual Acuity Ability Mean (All)" : visualAcuityAbilityAllMean,
                        "Flat Navigation Ability Mean (All)" : flatNavAbilityAllMean,
                        "Lava Ability Mean (All)" : lavaAbilityAllMean,
                        "Right Ability Mean (All)" : rightAbilityAllMean,
                        "Left Ability Mean (All)" : leftAbilityAllMean,
                        "Ahead Ability Mean (All)" : aheadAbilityAllMean,
                        "CVChick Ability Mean (All)" : CVChickAbilityAllMean,
                        "PCTB Ability Mean (All)" :  PCTBAbilityAllMean,
                        "Lights Out Ability Mean (All)" : lightsOutAbilityAllMean,
                        "Noise Parameter Mean (All)" : noiseParAllMean,
                        "Object Permanence Ability SD (All)": OPAbilityAllSD,
                        "Visual Acuity Ability SD (All)" : visualAcuityAbilityAllSD,
                        "Flat Navigation Ability SD (All)" : flatNavAbilityAllSD,
                        "Lava Ability SD (All)" : lavaAbilityAllSD,
                        "Right Ability SD (All)" : rightAbilityAllSD,
                        "Left Ability SD (All)" : leftAbilityAllSD,
                        "Ahead Ability SD (All)" : aheadAbilityAllSD,
                        "CVChick Ability SD (All)" : CVChickAbilityAllSD,
                        "PCTB Ability SD (All)" :  PCTBAbilityAllSD,
                        "Lights Out Ability SD (All)" : lightsOutAbilityAllSD,
                        "Noise Parameter SD (All)" : noiseParAllSD,
                        "Object Permanence Ability Min (All)": OPAbilityAllMin,
                        "Visual Acuity Ability Min (All)" : visualAcuityAbilityAllMin,
                        "Flat Navigation Ability Min (All)" : flatNavAbilityAllMin,
                        "Lava Ability Min (All)" : lavaAbilityAllMin,
                        "Right Ability Min (All)" : rightAbilityAllMin,
                        "Left Ability Min (All)" : leftAbilityAllMin,
                        "Ahead Ability Min (All)" : aheadAbilityAllMin,
                        "CVChick Ability Min (All)" : CVChickAbilityAllMin,
                        "PCTB Ability Min (All)" :  PCTBAbilityAllMin,
                        "Lights Out Ability Min (All)" : lightsOutAbilityAllMin,
                        "Noise Parameter Min (All)" : noiseParAllMin,
                        "Object Permanence Ability Max (All)": OPAbilityAllMax,
                        "Visual Acuity Ability Max (All)" : visualAcuityAbilityAllMax,
                        "Flat Navigation Ability Max (All)" : flatNavAbilityAllMax,
                        "Lava Ability Max (All)" : lavaAbilityAllMax,
                        "Right Ability Max (All)" : rightAbilityAllMax,
                        "Left Ability Max (All)" : leftAbilityAllMax,
                        "Ahead Ability Max (All)" : aheadAbilityAllMax,
                        "CVChick Ability Max (All)" : CVChickAbilityAllMax,
                        "PCTB Ability Max (All)" :  PCTBAbilityAllMax,
                        "Lights Out Ability Max (All)" : lightsOutAbilityAllMax,
                        "Noise Parameter Max (All)" : noiseParAllMax})
finalDF.to_csv("/content/drive/Shareddrives/comparative-object-permanence/finalResults.csv")


In [ ]:
with open('/content/drive/Shareddrives/comparative-object-permanence/finalResults.pkl', 'wb') as f:
      pickle.dump(finalDF, f)